We rarely get our data in just the form we want.  In this worksheet we will investigate some techniques for making relatively simple adjustments that help us get _from our input data into a data frame that is ready for us to use_.

We have some data in the file `iris.csv` which has been hand-transcribed from notebooks and contains a few errors.  To clean the data we will:
  * Load up the data
  * Understand the Data
  * Remove empty values
  * Fix format errors
  * Fix incorrect data
  * Remove duplicates

After this, we are ready to visualise the data (which we will learn more about next week).

In [ ]:
import pandas as pd

# load data - notice that iris data has no index, so we will use a fresh one
iris = pd.read_csv("data/iris.csv")
iris


Jupyter always gives us a preview of our data when we print it, but we can specifially ask for the `head` rows, the `tail` rows and `info` about any dataframe.  `info` in particular gives us very useful information about our data.  Pay close attention to the "non-null" count and the data type of each column.


In [ ]:
import pandas as pd

# load data - notice that iris data has no index, so we will use a fresh one
iris = pd.read_csv("data/iris.csv")
iris


In [ ]:
print(iris.head(2))
print("")
print(iris.tail(2))
print("")
print(iris.info())



The first issue that shows up is the missing value in "sepal width".  Every other column has 152 values, but this one has just 151.  Missing values in a table are called "null" values and they can mess with our analysis if we are not aware of them.  Sometimes we do want to leave them there, but often we want to exclude that data.

**YOU MUST NEVER MODIFY YOUR SOURCE DATA**

If there is "junk" in your input file, never make changes directly in the file, for the following reasons:
  * data is often audited and modification of official data could be an infringement
  * one person's junk is another person's treasure
  * pandas can adjust the data for you easily so you can have a "clean" version without adjusting the original

Lets start by finding that null value.  We know it is in the "sepal width" series


In [ ]:
iris["sepal width"]

we can't see it in the preview, but pandas can filter a series to keep only the null values, but do that, we will need to understand indexing with masks.   

Firstly, we get the "sepal width" series from the data frame.  Then we call a function on that series which will convert all values to true or false.  If the original value was not null, we will get false, if it was null, we get true.  You will have to believe me, but there is a single "True" in there.

In [ ]:
mask = iris["sepal width"].isnull()

print(mask.info())

mask

Recall that we call a series that is all booleans a "mask". Only indexes with a "True" in the mask result in a row being included in the result.

In [ ]:
iris[mask]

and thus we can see our problem.  There is a number missing in row 128.  Check the original dataframe to see for yourself.  We can also use `loc` or `iloc` to see the rows in this vicinity now we have identified the problem

In [ ]:
iris.loc[120:130,:]

# Exercise - wide petals

Identify all rows where the "petal width" is greater than 5.  How many are there?

In [ ]:
print("put your answer in this code block")

mask = iris["petal width"] > 5
count = len(iris[mask].axes[0])

print(count)

## Fixing the error

What do to about the error is up to you.  You should create a "clean" data frame, separate to the other one regardless of your decision.  Possible choices are:
  * remove that whole row
  * remove that whole column
  * choose a value for the missing entry

We will demonstrate each.


In [ ]:
# remove that row

# we need the opposite mask. A trick to do this is to perform the "equals false" operation :)
mask2 = mask == False

# we can then use this as a mask to get only the rows we want.
clean_iris = iris[mask2]
clean_iris

In [ ]:
# alternative!  Since we know exactly what row to drop, we can use the drop function

clean_iris = iris.drop(index=126)
clean_iris

In [ ]:
# alternative - dropna does _all_ the hard work for us
clean_iris = iris.dropna()
clean_iris

In [ ]:
# remove whole column with drop

clean_iris = iris.drop(columns="sepal width")
clean_iris

In [ ]:
# choose a value for the missing entry with `fillna`
clean_iris = iris.fillna(0)
clean_iris.loc[125:128]

We used a couple of helper functions. [`dropna` will remove any rows with empty data](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) while [`fillna` will replace any empty values with some other value](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

I tend not to fill in missing values, I tend to drop the whole row - that should be your default option but don't do it without paying attention to what is dropped and why.  Too many people just apply `dropna` without thinking.

# Exercise - find errors

Identify the erroneous data in the `class` column and remove that row.

**Advanced** Imagine you had not seen the error, what type of pandas code could you construct to find such an error.  I would suggest extracting that column as a series, get all the unique values in the series (`drop_duplicates` will help).

In [ ]:
print("put your solution here")

unique = iris["class"].drop_duplicates()

#invalid row below
#124                6.3

#remove index 124
clean_iris = iris.drop(index=124)

#check again

unique = clean_iris["class"].drop_duplicates()

unique

# Exercise - fishing

**Advanced** Find any other erroneous data and fix it.

In [ ]:
print("put your solution here")

# check data - there is a max petal width  = 22! where the mean = 1.3..
clean_iris.describe()

# find error
mask = clean_iris["petal width"] == 22
drop_index = clean_iris[mask].index

cleaner_iris = clean_iris.drop(index = drop_index)

#petal width max now = 2.5
cleaner_iris.describe()

# Exercise

There is a very useful method available on dataframes called "describe".  Below is an example of its use.

In [ ]:
# data/rainfall/lithgow.csv was not provided
# replaced with a different CSV
lithgow = pd.read_csv("data/rainfall/IDCJAC0009_047045_1800_Data.csv")
lithgow.describe()

Experiment with this method on data you know well, [check the documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html).  How do you think this method can help you find erroneous data in your DataFrames?

# Concept Summary
  * We can index into a table with a mask
  * We can use boolean operators to create useful masks
  * `head`, `tail`, and `info` are useful for learning about your data
  * `dropna`, `drop_duplicates`, `duplicates`, `fillna` are usefull for cleaning your data

# Python concepts
  * `head`, `tail`, `info`, `dropna`, `drop_duplicates`, `duplicates`, `fillna` are all _methods_ on the data frame object
  * most of the methods we used returned entirely fresh values which we needed to capture in a new variable.  The original data frame was not changed by methods like `fillna`.
  * the `drop` method of a dataframe can take different parameters to do different things.  `index=` will drop a row, `columns=` will drop a column.  Both versions return a whole new DataFrame, leaving the original untouched.
